In [1]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [2]:
data = sm.datasets.fair.load_pandas().data

data["affairs"] = np.ceil(data["affairs"])
data[:6]

rate_marriage   age  yrs_married  children  religious  educ  occupation  \
0            3.0  32.0          9.0       3.0        3.0  17.0         2.0   
1            3.0  27.0         13.0       3.0        1.0  14.0         3.0   
2            4.0  22.0          2.5       0.0        1.0  16.0         3.0   
3            4.0  37.0         16.5       4.0        3.0  16.0         5.0   
4            5.0  27.0          9.0       1.0        1.0  14.0         3.0   
5            4.0  27.0          9.0       0.0        2.0  14.0         3.0   

   occupation_husb  affairs  
0              5.0      1.0  
1              4.0      4.0  
2              5.0      2.0  
3              5.0      1.0  
4              4.0      5.0  
5              4.0      5.0

In [21]:
data2 = data.copy()
data2["const"] = 1
dc = (
    data2["affairs rate_marriage age yrs_married const".split()]
    .groupby("affairs rate_marriage age yrs_married".split())
    .count()
)
dc.reset_index(inplace=True)
dc.rename(columns={"const": "freq"}, inplace=True)
print(dc.shape)
dc.head()

(476, 5)


affairs  rate_marriage   age  yrs_married  freq
0      0.0            1.0  17.5          0.5     1
1      0.0            1.0  22.0          2.5     3
2      0.0            1.0  27.0          2.5     1
3      0.0            1.0  27.0          6.0     5
4      0.0            1.0  27.0          9.0     1

In [30]:
gr = data["affairs rate_marriage age yrs_married".split()].groupby(
    "rate_marriage age yrs_married".split()
)
df_a = gr.agg(["mean", "sum", "count"])
df_a.reset_index(inplace=True)

df_a

rate_marriage   age yrs_married   affairs            
                                         mean   sum count
0             1.0  17.5         0.5  0.000000   0.0     1
1             1.0  22.0         2.5  3.900000  39.0    10
2             1.0  27.0         2.5  3.400000  17.0     5
3             1.0  27.0         6.0  0.900000   9.0    10
4             1.0  27.0         9.0  1.333333   4.0     3
..            ...   ...         ...       ...   ...   ...
125           5.0  42.0         6.0  0.000000   0.0     2
126           5.0  42.0         9.0  0.000000   0.0     3
127           5.0  42.0        13.0  0.666667   8.0    12
128           5.0  42.0        16.5  0.166667   5.0    30
129           5.0  42.0        23.0  0.328413  89.0   271

[130 rows x 6 columns]

## Example


In [ ]:
data2 = data.copy()
data2["const"] = 1
dc = (
    data2["affairs rate_marriage age yrs_married const".split()]
    .groupby("affairs rate_marriage age yrs_married".split())
    .count()
)
dc.reset_index(inplace=True)
dc.rename(columns={"const": "freq"}, inplace=True)

In [71]:
## creating dataframe

data = pd.DataFrame()

data['uin'] = np.array(range(50))

data['group'] = np.random.binomial(1, 0.2, 50)

for i,p in zip(range(1,6), [0.8, 0.6, 0.5, 0.1, 0.05]):
    data[str(i)] = np.random.binomial(1, p, 50)

data = pd.melt(data, id_vars = 'uin	group'.split()).drop('uin', axis=1)

data.columns = 'group time outcome'.split()
data2 = data.copy()

data = data.groupby('group time'.split()).agg(["mean", "sum", "count"]).reset_index()
data.droplevel(0,axis=1)
data.columns = 'group time out_mean out_sum out_count'.split()

data['time'] = data['time'].astype(float)

data[:6]

group  time  out_mean  out_sum  out_count
0      0   1.0     0.875       35         40
1      0   2.0     0.550       22         40
2      0   3.0     0.525       21         40
3      0   4.0     0.125        5         40
4      0   5.0     0.025        1         40
5      1   1.0     0.800        8         10

In [77]:
data2["const"] = 1
dc = (
    data2["group	time	outcome const".split()]
    .groupby("group	time	outcome".split())
    .count()
)
dc.reset_index(inplace=True)
dc.rename(columns={"const": "freq"}, inplace=True)

dc['time'] = dc['time'].astype(float)
dc

group  time  outcome  freq
0       0   1.0        0     5
1       0   1.0        1    35
2       0   2.0        0    18
3       0   2.0        1    22
4       0   3.0        0    19
5       0   3.0        1    21
6       0   4.0        0    35
7       0   4.0        1     5
8       0   5.0        0    39
9       0   5.0        1     1
10      1   1.0        0     2
11      1   1.0        1     8
12      1   2.0        0     5
13      1   2.0        1     5
14      1   3.0        0     4
15      1   3.0        1     6
16      1   4.0        0     9
17      1   4.0        1     1
18      1   5.0        0    10

In [78]:
glm = smf.glm(
    "outcome ~ C(group) + time + C(group)*time",
    data=dc,
    family=sm.families.Poisson(),
    freq_weights=np.asarray(dc["freq"]),
)
res_f = glm.fit()
print(res_f.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                outcome   No. Observations:                   19
Model:                            GLM   Df Residuals:                      246
Model Family:                 Poisson   Df Model:                            3
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -167.02
Date:                Mon, 08 Nov 2021   Deviance:                       126.04
Time:                        14:52:28   Pearson chi2:                     124.
No. Iterations:                     5   Pseudo R-squ. (CS):             0.9486
Covariance Type:            nonrobust                                         
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              0.5475      0

In [79]:
data

group  time  out_mean  out_sum  out_count
0      0   1.0     0.875       35         40
1      0   2.0     0.550       22         40
2      0   3.0     0.525       21         40
3      0   4.0     0.125        5         40
4      0   5.0     0.025        1         40
5      1   1.0     0.800        8         10
6      1   2.0     0.500        5         10
7      1   3.0     0.600        6         10
8      1   4.0     0.100        1         10
9      1   5.0     0.000        0         10

In [80]:

glm = smf.glm(
    "out_mean ~ C(group) + time + C(group)*time",
    data=data,
    family=sm.families.Poisson(),
    var_weights=np.asarray(data["out_count"]),
)
res_e = glm.fit()
print(res_e.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:               out_mean   No. Observations:                   10
Model:                            GLM   Df Residuals:                        6
Model Family:                 Poisson   Df Model:                            3
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -149.12
Date:                Mon, 08 Nov 2021   Deviance:                       13.920
Time:                        14:53:12   Pearson chi2:                     13.7
No. Iterations:                     5   Pseudo R-squ. (CS):             0.9964
Covariance Type:            nonrobust                                         
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              0.5475      0